In [1]:
import pandas as pd
import numpy as np
import os
import sys
import Mixture
import multiprocessing
import matplotlib.pyplot as plt

#Library to read RDS format
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

In [2]:
print('Num cores: ' + str(multiprocessing.cpu_count()))

cores = 4
iters = 4
output = 'NewmanFL_TIL10_500pops'

if cores > multiprocessing.cpu_count():
    cores = multiprocessing.cpu_count()

Num cores: 4


In [3]:
# Read xlsx files
X = pd.read_excel('data/TIL10_signature.xlsx', sheet_name = 0) 

In [4]:
# Read xlsx files
Y = pd.read_excel('data/NewmanFL.xlsx', sheet_name = 0) 

In [5]:
# Run Mixer Function
if __name__ == '__main__':
    result, pValues = Mixture.Mixture(X, Y , cores, iters, output)

Running mixer with subjects (Count: 15)...
Normalizing data
Processing...
--------------------------------------------------
Subject: GSM1587831_FL_lymph_node_biopsy_untreated_1063.CEL.gz Nro. Processor: <_MainProcess(MainProcess, started)>
--------------------------------------------------
--------------------------------------------------
Subject: GSM1587832_FL_lymph_node_biopsy_untreated_1080.CEL.gz Nro. Processor: <_MainProcess(MainProcess, started)>
--------------------------------------------------
--------------------------------------------------
Subject: GSM1587833_FL_lymph_node_biopsy_untreated_575.CEL.gz Nro. Processor: <_MainProcess(MainProcess, started)>
--------------------------------------------------
--------------------------------------------------
Subject: GSM1587834_FL_lymph_node_biopsy_untreated_581.CEL.gz Nro. Processor: <_MainProcess(MainProcess, started)>
--------------------------------------------------
Iter: 1 Subject: GSM1587834_FL_lymph_node_biopsy_untreat

In [10]:
type(result)

pandas.core.frame.DataFrame

In [6]:
# Getting Real Betas Matrix
vector = [x[0] for x in subject]
columns = ['V' + str(x+1) for x in range(len(subject))]
Betas = pd.DataFrame(np.column_stack(vector), columns=columns)

NameError: name 'subject' is not defined

In [ ]:
def blandAltman (betas, result):
    betahat = result.Subjects[0].MIXprop[0].T.to_numpy(copy=True)
    betahat = betahat.flatten()
    
    betasim = betas.to_numpy(copy=True)
    betasim = betasim.flatten()
    
    df = pd.DataFrame(np.transpose([betahat, betasim, (betahat - betasim)]), columns = ['betahat', 'betasim', 'difs'])
    summary = pd.DataFrame([[np.mean(df.difs),np.std(df.difs),np.min(df.difs),np.max(df.difs)]], columns = ['mn', 'sd', 'mi', 'mx'])
    return df, summary
       

In [ ]:
ba, summary = blandAltman (Betas, result)

In [ ]:
def blandAltamGraph(ba, summary):
    fig, ax = plt.subplots(figsize=(16,8))
    ax.plot(ba.betasim, ba.difs, 'k.')
    plt.xlabel('Beta Sim')
    plt.ylabel('Difs')
    ax.axhline(summary.mn[0])
    ax.axhline(summary.mn[0]+2*summary.sd[0])
    ax.axhline(summary.mn[0]-2*summary.sd[0])

In [ ]:
blandAltamGraph(ba, summary)